In [1]:
import numpy as np 
import pandas as pd 
import geopandas as gpd 
import folium 
import datetime
import json
import branca
import statistics
import random as rd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import requests


In [2]:
#récupère la liste des stops points doublons, modification de leur access par la moyenne de leur accessbilité  avec ajout d'une nouvelle colonne 'new_access'
def access_doublons(str_chemin_csv):
    list_index_doublons = []
    df = pd.read_csv(str_chemin_csv, sep=";")

    for k in range(len(df)):
        if len(df[df['Coord'] == df.loc[k]['Coord']])>1:
            index = df[df['Coord'] == df.loc[k]['Coord']].index.values
            tmp = [index[j] for j in range(len(index))]
            if tmp not in list_index_doublons :
                list_index_doublons.append(tmp) 
    list_idx = []
    for j in range(len(list_index_doublons)) :
        list_idx.append(list_index_doublons[j][0])
        list_idx.append(list_index_doublons[j][1])

    list_new_access = [0.0]*len(df)
    idx = 0
    for k in range(len(df)):
        if k in list_idx :
            l = len(df[df['Coord'] == df.loc[k]['Coord']])
            s = np.sum(list(df.loc[list_index_doublons[idx]]['Access']))
            m = np.round(s/l,2)
            list_new_access[list_index_doublons[idx][0]] = m 
            list_new_access[list_index_doublons[idx][1]] = m
            for ele in range(l):
                list_idx.remove(list_index_doublons[idx][ele])
            idx+= 1
        elif list_new_access[k] == 0.0 : 
            list_new_access[k] = df.loc[k]['Access']

    df['new_access']=list_new_access
    #print(list_index_doublons)

    return df

In [3]:
## Création des df, attention les df avce 150 et 200 points sont calculés avec 5 ss processus et non 10 donc data ordonnées différement 

data_stop_0930_25 = access_doublons('./25points/csv/merged_1000SP_09h30_25points.csv')
data_stop_0930_50 = access_doublons('./50points/csv/merged_1000SP_09h30_50points.csv') 
data_stop_0930_100 = access_doublons('./100points/csv/merged1000SP_09h30_100points.csv') 
data_stop_0930_150 = access_doublons('./150points/csv/merged1000SP_09h30_150points.csv')
data_stop_0930_200 = access_doublons('./200points/csv/merged1000SP_09h30_200points.csv')


In [4]:
## on va donc réorganiser les df 150 et 200

def reorder_df(df): #reorder par rapport à data_stop_0930_100
    list_data = []

    for k in range(len(df)):
        data = df[(df['Coord']==data_stop_0930_100.loc[k]['Coord'])]
        if len(df[(df['Coord']==data_stop_0930_100.loc[k]['Coord'])])<=1:
            list_data.append(data)
        else : #lorsqu'il y a un doublon on ne peut les réordonner d'une manière certaine donc on prend la première ligne trouvée pas d'importance sur new_access
            data_doublon = df[(df['Coord']==data_stop_0930_100.loc[k]['Coord'])][:1]
            list_data.append(data_doublon)
    df_reorder = pd.concat(list_data)
    return df_reorder

data_stop_0930_150 = reorder_df(data_stop_0930_150)
data_stop_0930_200 = reorder_df(data_stop_0930_200)

In [5]:
def conversion_temps_unique(d):
    hours = int(d//3600)  
    if 10>hours>=0 :
        min = int(np.round(d%3600//60))
        sec = int(np.round(d%3600%60))
        if min<10 and sec<10:
            return '0'+str(hours)+':0'+str(min)+':0'+str(sec)
        elif min>=10 and sec>=10:
            return '0'+str(hours)+':'+str(min)+':'+str(sec)
        elif min>=10 and sec<10 :
            return '0'+str(hours)+':'+str(min)+':0'+str(sec)
        else :
            return '0'+str(hours)+':0'+str(min)+':'+str(sec)
    return '>=10h'

In [6]:
def conversion_temps_access(df):
    list_access_s = list(df['new_access'])
    list_access,list_access_min = [], []
    s1,s10 = 0,0
    for k in range(len(list_access_s)):
        list_access_min.append(np.round(list_access_s[k]/60,2))
        hours = int(list_access_s[k]//3600)
        if 10>hours>=0 :
            if hours >1 :
                #print(k,'... grande accessibilité')
                s1+=1
            min = int(np.round(list_access_s[k]%3600//60))
            sec = int(np.round(list_access_s[k]%3600%60))
            if min<10 and sec<10:
                list_access.append('0'+str(hours)+':0'+str(min)+':0'+str(sec))
            elif min>=10 and sec>=10:
                list_access.append('0'+str(hours)+':'+str(min)+':'+str(sec))
            elif min>=10 and sec<10 :
                list_access.append('0'+str(hours)+':'+str(min)+':0'+str(sec))
            else :
                list_access.append('0'+str(hours)+':0'+str(min)+':'+str(sec))
        elif hours>=10 :
            #print(k,'... grande accessibilité')
            s10+=1
            min = int(np.round(list_access_s[k]%3600//60))
            sec = int(np.round(list_access_s[k]%3600%60))
            if min<10 and sec<10:
                list_access.append(str(hours)+':0'+str(min)+':0'+str(sec))
            elif min>=10 and sec>=10:
                list_access.append(str(hours)+':'+str(min)+':'+str(sec))
            elif min>=10 and sec<10 :
                list_access.append(str(hours)+':'+str(min)+':0'+str(sec))
            else :
                list_access.append(str(hours)+':0'+str(min)+':'+str(sec))
        else : 
            list_access.append('00:00:00')
    print(s1, ' SP ont une access > 1h')
    print(s10, ' SP ont une access >= 10h')
    df['access_min'] = list_access_min
    df['access_h_m_s']= list_access

In [7]:
def coord_gps(df):

    Coord_gps_list, Latitude_list, Longitude_list = [], [], []

    for coord in list(df['Coord']):
        index = coord.find(',')
        lat = coord[:index]
        lon = coord[index+1:]
        Longitude_list.append(float(lon))
        Latitude_list.append(float(lat))
        Coord_gps_list.append([lat,lon])

    df['Coord_gps'] = Coord_gps_list
    df['Longitude'] = Longitude_list
    df['Latitude'] = Latitude_list

coord_gps(data_stop_0930_25)
coord_gps(data_stop_0930_50)
coord_gps(data_stop_0930_100)
coord_gps(data_stop_0930_150)
coord_gps(data_stop_0930_200)

In [8]:
list_df_0930 = [data_stop_0930_25,data_stop_0930_50,data_stop_0930_100,data_stop_0930_150,data_stop_0930_200]
data_comparaison_0930 = pd.concat(list_df_0930)

In [9]:
conversion_temps_access(data_comparaison_0930)

638  SP ont une access > 1h
0  SP ont une access >= 10h


In [10]:
#création CSV modifiés
data_comparaison_0930.to_csv('./stops_concat_1000SP_0930.csv', sep = ';',index=False) 

data_stop_0930_25.to_csv('./25points/csv/1000SP_09h30_25points.csv', sep = ';',index=False)
data_stop_0930_50.to_csv('./50points/csv/1000SP_09h30_50points.csv', sep = ';',index=False)
data_stop_0930_100.to_csv('./100points/csv/1000SP_09h30_100points.csv', sep = ';',index=False)
data_stop_0930_150.to_csv('./150points/csv/1000SP_09h30_150points.csv', sep = ';',index=False)
data_stop_0930_200.to_csv('./200points/csv/1000SP_09h30_200points.csv', sep = ';',index=False)

In [11]:
data_stop_0830 = access_doublons('./100points/csv/merged1000SP_08h30_100points.csv')
data_stop_09 = access_doublons('./100points/csv/merged1000SP_09h00_100points.csv') 
data_stop_0930 = access_doublons('./100points/csv/merged1000SP_09h30_100points.csv')
data_stop_10 = access_doublons('./100points/csv/merged1000SP_10h00_100points.csv') 
data_stop_1030 = access_doublons('./100points/csv/merged1000SP_10h30_100points.csv')

#
coord_gps(data_stop_0830)
coord_gps(data_stop_09)
coord_gps(data_stop_0930)
coord_gps(data_stop_10)
coord_gps(data_stop_1030)

conversion_temps_access(data_stop_0830)
conversion_temps_access(data_stop_09)
conversion_temps_access(data_stop_0930)
conversion_temps_access(data_stop_10)
conversion_temps_access(data_stop_1030)
#

87  SP ont une access > 1h
0  SP ont une access >= 10h
191  SP ont une access > 1h
0  SP ont une access >= 10h
125  SP ont une access > 1h
0  SP ont une access >= 10h
132  SP ont une access > 1h
0  SP ont une access >= 10h
120  SP ont une access > 1h
0  SP ont une access >= 10h


In [12]:
data_stop_0830.head()

,Name,Coord,Access,time,new_access,Coord_gps,Longitude,Latitude,access_min,access_h_m_s
0,Jean Jaurès,"48.974702,1.803076",6420.16,03-08-2021 08:30,6420.16,"[48.974702, 1.803076]",1.803076,48.974702,107.00,01:47:00
1,GARE DE ST GRATIEN,"48.964243,2.285417",3521.32,03-08-2021 08:30,3521.32,"[48.964243, 2.285417]",2.285417,48.964243,58.69,00:58:41
2,Pierre Stassinet,"48.78268,1.959276",6981.49,03-08-2021 08:30,6981.49,"[48.78268, 1.959276]",1.959276,48.782680,116.36,01:56:21
3,Quartier de Lattre,"48.533199,3.372171",0.00,03-08-2021 08:30,0.00,"[48.533199, 3.372171]",3.372171,48.533199,0.00,00:00:00
4,Croix de Vernailles,"48.44603,2.166975",11716.80,03-08-2021 08:30,11716.80,"[48.44603, 2.166975]",2.166975,48.446030,195.28,03:15:17


In [13]:
def postcode_dep(df):

    list_postcode = []
    list_dep = []

    for k in range(len(df)):
        res = requests.get('https://api-adresse.data.gouv.fr/reverse/?lon='+list(df['Coord_gps'])[k][1]+'&lat='+list(df['Coord_gps'])[k][0])
        data = res.json()
        if len(data['features'])!=0:
            postcode = data['features'][0]['properties']['postcode'][:2]
            list_postcode.append(postcode)
            if postcode=='75' :
                list_dep.append('Paris')
            elif postcode=='77' :
                list_dep.append('Seine-et-Marne')
            elif postcode=='78' :
                list_dep.append('Yvelines')
            elif postcode=='91' :
                list_dep.append('Essonne')
            elif postcode=='92' :
                list_dep.append('Hauts-de-Seine')
            elif postcode=='93' :
                list_dep.append('Seine-Saint-Denis')
            elif postcode=='94' :
                list_dep.append('Val-de-Marne')
            elif postcode=='95' :
                list_dep.append('Val-d\'Oise')
            else : 
                print(postcode,' : postcode pas en Ile de France')
                list_dep.append('Erreur postcode pas en Ile de France')
        else : 
            print('Erreur API, postcode non disponible')
            list_postcode.append(0)
            list_dep.append('Erreur API')

    df['postcode']=list_postcode
    df['dep']=list_dep

postcode_dep(data_stop_0830) #pour diminuer grandement le temps on peut simplement le faire sur un df puis recopier les colonnes dep et postcode dans les autres df car même ordre...
postcode_dep(data_stop_09)
postcode_dep(data_stop_0930)
postcode_dep(data_stop_10)
postcode_dep(data_stop_1030)

27  : postcode pas en Ile de France
Erreur API, postcode non disponible
Erreur API, postcode non disponible
28  : postcode pas en Ile de France
28  : postcode pas en Ile de France
45  : postcode pas en Ile de France
27  : postcode pas en Ile de France
Erreur API, postcode non disponible
Erreur API, postcode non disponible
28  : postcode pas en Ile de France
28  : postcode pas en Ile de France
45  : postcode pas en Ile de France
27  : postcode pas en Ile de France
Erreur API, postcode non disponible
Erreur API, postcode non disponible
28  : postcode pas en Ile de France
28  : postcode pas en Ile de France
45  : postcode pas en Ile de France
27  : postcode pas en Ile de France
Erreur API, postcode non disponible
Erreur API, postcode non disponible
28  : postcode pas en Ile de France
28  : postcode pas en Ile de France
45  : postcode pas en Ile de France
27  : postcode pas en Ile de France
Erreur API, postcode non disponible
Erreur API, postcode non disponible
28  : postcode pas en Ile de

In [14]:
list_data = [data_stop_0830,data_stop_09,data_stop_0930,data_stop_10,data_stop_1030]
data_concat_0830_1030 = pd.concat(list_data)

In [15]:
list_time_min=[]

for k in range(len(data_concat_0830_1030)):
    if list(data_concat_0830_1030['time'])[k]=='03-08-2021 08:30':
        list_time_min.append(510)
    elif list(data_concat_0830_1030['time'])[k]=='03-08-2021 09:00':
        list_time_min.append(540)
    elif list(data_concat_0830_1030['time'])[k]=='03-08-2021 09:30':
        list_time_min.append(570)
    elif list(data_concat_0830_1030['time'])[k]=='03-08-2021 10:00':
        list_time_min.append(600)
    elif list(data_concat_0830_1030['time'])[k]=='03-08-2021 10:30':
        list_time_min.append(630)
        
data_concat_0830_1030['time_min'] = list_time_min

In [16]:
#data_concat_0830_1030[data_concat_0830_1030['dep']=='Erreur postcode pas en Ile de France']

In [17]:
conversion_temps_access(data_concat_0830_1030)

655  SP ont une access > 1h
0  SP ont une access >= 10h


In [18]:
from collections import Counter

Counter(list(data_concat_0830_1030[data_concat_0830_1030['access_h_m_s']>'01:00:00']['dep']))

Counter({'Yvelines': 708,
         'Essonne': 590,
         'Seine-et-Marne': 499,
         'Seine-Saint-Denis': 297,
         'Hauts-de-Seine': 324,
         "Val-d'Oise": 424,
         'Val-de-Marne': 240,
         'Paris': 163,
         'Erreur API': 10})

In [19]:
liste_var = [0]*1000

for k in range(1000):
    liste_var[k] = statistics.pstdev(data_concat_0830_1030[data_concat_0830_1030['Name'] == list(data_concat_0830_1030['Name'])[k]]['Access'])

data_concat_0830_1030['Variance_access'] = liste_var*5 #à vérifier doublons, même nom et diff coord gps..

In [20]:
#création CSV modifiés

data_stop_0830.to_csv('/home/thd/idf-accessibility/ponctuel/100/csv/1000SP_0830.csv', sep = ';',index=False)
data_stop_09.to_csv('/home/thd/idf-accessibility/ponctuel/100/csv/1000SP_09.csv', sep = ';',index=False)
data_stop_0930.to_csv('/home/thd/idf-accessibility/ponctuel/100/csv/1000SP_0930.csv', sep = ';',index=False)
data_stop_10.to_csv('/home/thd/idf-accessibility/ponctuel/100/csv/1000SP_10.csv', sep = ';',index=False)
data_stop_1030.to_csv('/home/thd/idf-accessibility/ponctuel/100/csv/1000SP_1030.csv', sep = ';',index=False)
data_concat_0830_1030.to_csv('/home/thd/idf-accessibility/ponctuel/100/csv/stops_concat_1000SP_0830_1030.csv', sep = ';',index=False)